This is a demo notebook for the [bcbcpy](https://github.com/aheritianad/BootCamp-BlockChain-and-Python/) package.

In [1]:
try:
    import bcbcpy
except ModuleNotFoundError:
    import os, sys
    sys.path.append(os.path.abspath("../src/"))
    import bcbcpy

# Cryptography


In [2]:
from bcbcpy import crypto

## Hashing


### `Hash` function


$$\mathfrak{h}: X \to H$$

- $\mathfrak{h}(x) = \mathfrak{h}(y)$ *only if* $x=y$.
- given $h\in H$, it is *not easy* to find $x\in X$ such that $\mathfrak{h}(x) = h$.

In some sense, $\mathfrak{h}(x)$ is a signature of $x$.

In [3]:
text_to_be_hashed = "Hello World"
first_hash = crypto.hash_function(text_to_be_hashed, hash_name=crypto.HashName.sha256)
first_hash


'1370eeaaba7a6c7a234b1f82cc3b6d013a0088fd5e16408300f05b28b0015463'

In [4]:
crypto.hash_function(text_to_be_hashed + "!")


'e59f8bdf1305e382a4919ccefd613d3eebae612aa4c443f3af2d65663de3b075'

In [5]:
crypto.hash_function("Hello World!")

'e59f8bdf1305e382a4919ccefd613d3eebae612aa4c443f3af2d65663de3b075'

In [6]:
long_text = """
Hello everyone!

Welcome to this tutorial.
I promise that I will do my best to help you.

It will be interactive, so please participate as much as you can.

Please do not hesitate to ask if there you have questions.

Hope you will enjoy it!

Cheers!

Heritiana.
"""


In [7]:
hash_long = crypto.hash_function(long_text)
hash_long


'0c8359860bef1b26f811bf8448c546c2241c7c695f9b8ee76c1a8bf2d5b2dff3'

In [8]:
len(hash_long) == len(first_hash)


True

### Validation


In [9]:
difficulty = 4
crypto.is_valid_hash(first_hash, difficulty)


False

In [10]:
fake_hash = "0000" + first_hash[4:]
fake_hash

'0000eeaaba7a6c7a234b1f82cc3b6d013a0088fd5e16408300f05b28b0015463'

In [11]:
crypto.is_valid_hash(fake_hash, difficulty)


True

### Nonce


In [12]:
second_hash, nonce = crypto.hash_nonce_initializer(
    text_to_be_hashed, difficulty=difficulty
)
second_hash

'0000bfe6af4232f78b0c8eba37a6ba6c17b9b8671473b0b82305880be077edd9'

In [13]:
nonce


107105

In [14]:
crypto.is_valid_hash(second_hash, difficulty)


True

In [15]:
third_hash = crypto.hash_function(text_to_be_hashed, nonce=nonce)
third_hash


'0000bfe6af4232f78b0c8eba37a6ba6c17b9b8671473b0b82305880be077edd9'

In [16]:
second_hash == third_hash


True

In [17]:
forth_hash = crypto.hash_function(text_to_be_hashed, nonce, nonce="")  # type:ignore
second_hash == forth_hash

True

### Q: What if such `nonce` does not exist?


**Think about it.**


## Crypto


### `Encryption` & `Decryption`


$$f\circ f^{-1} (x) = f^{-1}\circ f(x) = x,\ \forall x$$


In [18]:
encrypt = lambda x: 2 * x - 1
decrypt = lambda x: 0  # TODO to fill

In [19]:
import random
try:
    for i in range(20):
        x = random.randint(-5000, 5000)
        assert (
            encrypt(decrypt(x)) == decrypt(encrypt(x)) == x
        ), f"Oops! fail at {i+1}-th attempt for x = {x}."
except AssertionError as e:
    print(e)
else:
    print("Congrats! I passes the test.")

Oops! fail at 1-th attempt for x = -4998.


In [20]:
clear_message = (
    "This is a clear message to encrypt and decrypt. I add this to make it longer."
)

### Types


#### Key


$\mathcal{F}=\{ f_\theta \:\ \theta \in \Theta\}$

$\left(f_{\theta}\right) ^{-1} = f_{\theta^\prime}$ for some $\theta^\prime \in \Theta$

$\theta \in \Theta$ : key


#### Symmetric


Given $f_\theta$ or $\theta\in \Theta$ (thus $f_\theta$), one can construct *easily*  $\left(f_\theta\right)^{-1}$.

From Wikipedia (2023-05-01):

*Symmetric-key algorithms are algorithms for cryptography that use the same cryptographic keys for both the encryption of plaintext and the decryption of ciphertext. The keys may be identical, or there may be a simple transformation to go between the two keys.*

##### Caesar


**Idea**: translation

$N$ : number of character.  
key: $ \theta \in  \mathbb{Z}$  
function: $f_\theta: x \mod N \mapsto (x + \theta) \mod N$  
inverse: $\theta^\prime = -\theta$

In [21]:
caesar_key = crypto.CaesarKey.generate_key(max_length=2)
print(caesar_key)


CaesarKey(2)


In [22]:
crypto.CaesarKey.compute_inverse(6)

-6

In [23]:
encr_caesar = caesar_key.encrypt(clear_message)

print(
    f"""
original:
========
{clear_message}

Encrypted:
========
{encr_caesar}
"""
)


original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Encrypted:
Vjku"ku"c"engct"oguucig"vq"gpet{rv"cpf"fget{rv0"K"cff"vjku"vq"ocmg"kv"nqpigt0



In [24]:
decr = caesar_key.decrypt(encr_caesar)

print(
    f"""
original:
========
{clear_message}

Decrypted:
=========
{decr}
"""
)


original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Decrypted:
This is a clear message to encrypt and decrypt. I add this to make it longer.



##### Permutation


In [25]:
crypto.PermutationKey.compute_inverse([2, 3, 4, 1])

[4, 1, 2, 3]

###### **Idea1: Chunking**

Chunk input with a chunk size equal to permutation length and apply permutation on each chunks.

In [26]:
perm_key = crypto.PermutationKey.generate_key(length=3, n_runs=1)
print(perm_key)


PermutationKey([1, 3, 2], [1, 3, 2])


In [27]:
encr_perm = perm_key.encrypt(clear_message)

print(
    f"""
original:
========
{clear_message}

Encrypted:
========
{encr_perm}
"""
)


original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Encrypted:
Tihsi sa  lcera emsasg et oecnrpyta n ddcerpyt .Ia d dtihst om aek ti olnegr.



In [28]:
decr = perm_key.decrypt(encr_perm)
print(
    f"""
original:
========
{clear_message}

Decrypted:
=========
{decr}
"""
)


original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Decrypted:
This is a clear message to encrypt and decrypt. I add this to make it longer.



###### **Idea2: Sliding**

One can think about sliding the permutation like the following.

Suppose we have the following permutation
$$\theta: \begin{cases}1\mapsto 3\\ 2\mapsto 2\\ 3\mapsto 1\end{cases}$$
and input `abcdefgh`.

- `input = abcdefgh`
- `abc defgh` $\to$ `cba defgh`
- `c bad efgh` $\to$ `c dab efgh`
- `cd abe fgh` $\to$ `cd eba fgh`
- `cde baf gh` $\to$ `cde fab gh`
- `cdef abg h` $\to$ `cdef gba h`
- `cdefg bah` $\to$ `cdefg hab`
- `output = cdefghab`

⚠️: the same process does not bring back `input` from output  
It can be fixed by doing a tricky thing to the *previous output* to get the *final output*.  
You can think about it and its implementation.

##### Adding noise


In [29]:
from bcbcpy import utils


In [30]:
noisy_message = utils.add_noises(clear_message)
noisy_message


'T;hViGss .ias, ga- Qc`lweHajr, =m!eNs)s%a%g^e% 4t8oM keMn%cQrnyYpztI >a}n~d` 9dIeFc4rayEp[t:.{ >I  oaCdSda It(h{i#sw 1tpo/ Sm2a/k1eJ 1i2tM jlnoUn6gMe=rI.K'

In [31]:
noisy_message[::2]

'This is a clear message to encrypt and decrypt. I add this to make it longer.'

In [32]:
utils.remove_noises(noisy_message)


'This is a clear message to encrypt and decrypt. I add this to make it longer.'

In [33]:
encr_noisy = caesar_key.encrypt(noisy_message)
decr_noisy = caesar_key.decrypt(encr_noisy)
decr_without_noise = utils.remove_noises(decr_noisy)

print(
    f"""
\t\t--- CAESAR --
Original:
========
{clear_message}

Noisy message:
=============
{noisy_message}

Noisy Encrypted:
===============
{encr_noisy}

Noisy Decrypted:
===============
{decr_noisy}

Cleaned Decrypted:
=================
{decr_without_noise}
"""
)


		--- CAESAR --
Original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Noisy message:
T;hViGss .ias, ga- Qc`lweHajr, =m!eNs)s%a%g^e% 4t8oM keMn%cQrnyYpztI >a}n~d` 9dIeFc4rayEp[t:.{ >I  oaCdSda It(h{i#sw 1tpo/ Sm2a/k1eJ 1i2tM jlnoUn6gMe=rI.K

Noisy Encrypted:
V=jXkIuu"0kcu."ic/"SebnygJclt."?o#gPu+u'c'i`g'"6v:qO"mgOp'eStp{[r|vK"@c	p
fb";fKgHe6tc{Gr]v<0}"@K""qcEfUfc"Kv*j}k%uy"3vrq1"Uo4c1m3gL"3k4vO"lnpqWp8iOg?tK0M

Noisy Decrypted:
T;hViGss .ias, ga- Qc`lweHajr, =m!eNs)s%a%g^e% 4t8oM keMn%cQrnyYpztI >a}n~d` 9dIeFc4rayEp[t:.{ >I  oaCdSda It(h{i#sw 1tpo/ Sm2a/k1eJ 1i2tM jlnoUn6gMe=rI.K

Cleaned Decrypted:
This is a clear message to encrypt and decrypt. I add this to make it longer.



In [34]:
encr_noisy = perm_key.encrypt(noisy_message)
decr_noisy = perm_key.decrypt(encr_noisy)
decr_without_noise = utils.remove_noises(decr_noisy)

print(
    f"""
\t\t--- PERMUTATION ---
Original:
========
{clear_message}

Noisy message:
=============
{noisy_message}

Noisy Encrypted:
===============
{encr_noisy}

Noisy Decrypted:
===============
{decr_noisy}

Cleaned Decrypted:
=================
{decr_without_noise}
"""
)


		--- PERMUTATION ---
Original:
This is a clear message to encrypt and decrypt. I add this to make it longer.

Noisy message:
T;hViGss .ias, ga- Qc`lweHajr, =m!eNs)s%a%g^e% 4t8oM keMn%cQrnyYpztI >a}n~d` 9dIeFc4rayEp[t:.{ >I  oaCdSda It(h{i#sw 1tpo/ Sm2a/k1eJ 1i2tM jlnoUn6gMe=rI.K

Noisy Encrypted:
Th;VGis s.ais ,g-a cQ`wleaHj,r m=!Ness)%%age^%4 to8Mk enM%QcrynYzpt I>}and~`9 deIF4cryaE[pt.:{> I  oCaddSaI th({#is w1pto /S2mak/1Je i12Mt ljnUong6M=er.IK

Noisy Decrypted:
T;hViGss .ias, ga- Qc`lweHajr, =m!eNs)s%a%g^e% 4t8oM keMn%cQrnyYpztI >a}n~d` 9dIeFc4rayEp[t:.{ >I  oaCdSda It(h{i#sw 1tpo/ Sm2a/k1eJ 1i2tM jlnoUn6gMe=rI.K

Cleaned Decrypted:
This is a clear message to encrypt and decrypt. I add this to make it longer.



#### Asymmetric: Public-key cryptography


Knowing $f_\theta$ or $\theta\in \Theta$ (thus $f_\theta$) does **NOT** make the construction of $\left(f_\theta\right)^{-1}$ (thus $\theta^\prime$) *easy*.

From Wikipedia (2023-05-01):

*Public-key cryptography, or asymmetric cryptography, is the field of cryptographic systems that use pairs of related keys. Each key pair consists of a public key and a corresponding private key. Key pairs are generated with cryptographic algorithms based on mathematical problems termed one-way functions. Security of public-key cryptography depends on keeping the private key secret; the public key can be openly distributed without compromising security.*

##### Rivest–Shamir–Adleman aka RSA


$\Theta \approx \mathbb{N} \times \mathbb{N}$  
public key: $\theta = (n,d)$  
private key: $\theta^\prime = (n,e)$  
function: $f_{(n,a)}: x\mod n \mapsto x^a \mod n$

**Construction**:  

1. Generate two primes $p,q$.
2. Compute $n=p\times q$.
3. Compute $\varphi(n) = (p-1)*(q-1)$.
4. Find $d$ such that $\gcd\left( d, \varphi(n)\right) = 1$.
5. Find $e$ such that $de + u\varphi(n)=1$ for some $u\in \mathbb{Z}$.
6. Discard $p,q,\varphi(n)$ and $u$ *(for security)*.

Note: [4.] and [5.] can be done easily and at once with the extended Euclidean algorithm.

**:warning: Security**: number factorization complexity

For large composite number $n$, computing $\varphi(n)$ is not *easy* without knowing its prime factors.

In [35]:
rsa_key = crypto.RSAPairKeys.generate_pairs(16)
print(rsa_key)

RSAPairKeys((34823, 12917), HIDDEN_KEY)


In [36]:
rsa_pub, rsa_priv = rsa_key
rsa_pub, rsa_priv

((34823, 12917), (34823, 9629))

In [37]:
encr_rsa = rsa_key.encrypt(clear_message)
encr_rsa

'2\\vjTt<$(l`\n"1}@0<&D`rzgIT+Ses;6<pl0^Rf<+|w|$tDj+3YJ@"`84(2X`M5Fk_\'>4Q(n79(UB$'

In [38]:
decr_rsa = rsa_key.decrypt(encr_rsa)
decr_rsa


'This is a clear message to encrypt and decrypt. I add this to make it longer.'

##### Diffie–Hellman key exchange

1. The network will:
   - choose a shared group $(G,*)$,
   - an element $g\in G$,
   - and compute $ord(g) \in \mathbb{N}$ for which is the smallest where $ord(g)\cdot g := \underbrace{g*g*\cdots*g}_{ord(g) \text{ times}}$ is equal to the neutral element of $G$.
2. For each $i$ in the network:
   - Choose $\theta^\prime_i \in \mathbb{N}$ such that $\theta^\prime_i < ord(g)$.  
   - Compute $f_{\theta_i} = \theta^\prime_i \cdot g$.
   - Publish $f_{\theta_i}$ and keep $\theta^\prime_i$ secret.

If $i$ wants to send a message $m$ to $j$, he will proceed as follows:

1. retrieve $f_{\theta_j}$,
2. compute $P_{ij} := \theta_i \cdot f_{\theta_j}$,
3. encrypt $m$ with $P_{ij}$ (like the symmetric fashion),
4. send encrypted message $m_{ij}$ to $j$.

When $j$ receives $m_{ij}$, he will:

1. retrieve $f_{\theta_i}$,
2. compute $P_{ji} := \theta_j \cdot f_{\theta_i}$,
3. decrypt $m_{ij}$ with $P_{ji}$.
   
Note:
$$ \begin{align*}
P_{ij} 
&= \theta_i \cdot f_{\theta_j}&\\
&= \underbrace{f_{\theta_j}*\cdots*f_{\theta_j}}_{\theta_i \text{ times}}&\\
&= \underbrace{\left(\underbrace{g*\cdots*g}_{\theta_j \text{ times}}\right)*\cdots*\left(\underbrace{g*\cdots*g}_{\theta_j \text{ times}}\right)}_{\theta_i \text{ times}}&\\
&= \underbrace{g*\cdots*g}_{\theta_i\theta_j \text{ times}}&\\
&= \underbrace{\left(\underbrace{g*\cdots*g}_{\theta_j \text{ times}}\right)*\cdots*\left(\underbrace{g*\cdots*g}_{\theta_i \text{ times}}\right)}_{\theta_j \text{ times}}&\\
&= \underbrace{f_{\theta_j}*\cdots*f_{\theta_i}}_{\theta_j \text{ times}}&\\
&= \theta_j \cdot f_{\theta_i}&\\
&= P_{ji}&
\end{align*}$$

The following image summarizes the above processes.

<p style="text-align: center;">
<img src="https://upload.wikimedia.org/wikipedia/commons/4/46/Diffie-Hellman_Key_Exchange.svg">
</p>

[[source: wikimedia](https://upload.wikimedia.org/wikipedia/commons/4/46/Diffie-Hellman_Key_Exchange.svg)]

**⚠️ Security**: *discrete logarithm*

For some group ([Diffie-Hellman Group](https://www.ibm.com/docs/en/zos/2.3.0?topic=SSLTBW_2.3.0%2Fcom.ibm.tcp.ipsec.ipsec.help.doc%2Fcom%2Fibm%2Ftcp%2Fipsec%2Fipsec%2FPS_DT_Adv_PFS.CB_Initiate.html)), knowing the value of $g$ and $n \cdot g$ does not lead to $n$ *easily*.

In [39]:
# coming soon

##### Elliptic Curve Cryptography


[ECC](https://en.wikipedia.org/wiki/Elliptic-curve_cryptography) use the DH key exchange with Where the group $G$ is an [elliptic curve over finite fields](https://en.wikipedia.org/wiki/Elliptic_curve#Elliptic_curves_over_finite_fields).

In [40]:
# coming soon

### Communication & Security


#### Nodes generation

In [41]:
from bcbcpy.node import Node


In [42]:
key = crypto.RSAPairKeys.generate_pairs()
node = Node(key)
node.id

'user_147168'

In [43]:
node

In [44]:
class RepNode(Node):
    def __repr__(self) -> str:
        return f"{self.id} : {self._Node__keys}"  # type:ignore

In [45]:
akey = crypto.RSAPairKeys.generate_pairs()
alice = RepNode(akey, "Alice")
alice


Alice : RSAPairKeys((33491, 24511), HIDDEN_KEY)

In [46]:
bkey =  crypto.RSAPairKeys.generate_pairs()
bob = RepNode(username="Bob", keys=bkey)
bob

Bob : RSAPairKeys((34553, 11981), HIDDEN_KEY)

In [47]:
rkey = crypto.RSAPairKeys.generate_pairs()
random_guy = RepNode(rkey)
random_guy

user_518397 : RSAPairKeys((52109, 27097), HIDDEN_KEY)

#### Communication

In [48]:
sender = alice
receiver = bob

plain_message = (
    "Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)"
)

In [49]:
cipher = sender.encrypt(plain_message, key=receiver.pub)
cipher

'\nF}\taJRY8^P\n]Fn:lQ[sq[M6r"3Qs8]=p\\u= D>=K"Z2~rU"PuF?76hI+13YDws.CcCaNRJ\'doS3&`'

In [50]:
receiver.decrypt(cipher)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

#### Attack

##### Reading attack


In [51]:
hacker = random_guy

In [52]:
hacker.decrypt(cipher)


"!Dkjlsz&5MDl)%M^HZ$!}]&'`p(@Vn/7aC$NQ8A2VN\thRw]pXwv-Dl '>[lO6)z,q&_r '6mi$;l]n6"

In [53]:
hacker.encrypt(cipher, receiver.pub)


"'k@u?z^Bf{['(Z`~&L5\n,[ui}rrZ&]/HwE}#1.d,2h\nh#1V6cxVg;7L(i0d/y1txP(POq+Q) 89;q9"

##### Q: what if `receiver.pub == receiver.priv`?

In [54]:
_, receiver_priv = receiver._Node__keys
receiver_pub = receiver_priv # fake pub for testing

In [55]:
hacker.encrypt(cipher, receiver_pub)


'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

##### Sending attack


In [56]:
scam_message = (
    "Hello Bob! This is Alice. Please send me $1000 to this account: (0987654321)"
)

In [57]:
scam_cipher= hacker.encrypt(scam_message, key=receiver.pub)
scam_cipher


'\nF}\taJRY8^P\n]Fn:lQ[sq[M6r"3Qs8]=p\\u= D>=K"Z2~rU"PuF?76hI+13YDws.CcI:;X{j+&X<<r'

In [58]:
receiver.decrypt(scam_cipher)

'Hello Bob! This is Alice. Please send me $1000 to this account: (0987654321)'

### Authentication


**Problem**: no way for the `receiver` to recognize the *true* `sender`.

#### Encryption

In [59]:
sender.sign(plain_message) == sender.decrypt(plain_message) # encrypt with sender.priv

True

In [60]:
signed_message = sender.sign(plain_message) 
cipher_signed = sender.encrypt(signed_message, key=receiver.pub)
cipher_signed

'/>?E1(V\tiR!aS%S)R6ZZA*0^paZkKOSDiI,p@gN[.j+WqQtp>7bl\\v_1Wy3\n:@\n(hLp8N(\nb/4Cq%B'

In [61]:
with_noise = utils.add_noises(plain_message)
signed_with_noise = sender.sign(with_noise) 
cipher_signed_with_noise = sender.encrypt(signed_with_noise, key=receiver.pub)
cipher_signed_with_noise

'e8#<)CbbI55\'7]zJuj\tC21!&t81>"&u<!7#O9DK|Px3:ML6J)kB]6F>nfd]LD#(k*:8y?WkX6doR@<5Xo81A"%#w/D$:Y4i7?i^M.i(01Fwpb!@*iV=`?WP[ozb1UfskPrh,TCCl]:BSTCLJ8+86B>W#X'

##### Shortcut

In [62]:
cipher_signed == sender.sends(plain_message, _to=receiver.pub)

True

In [63]:
cipher_signed == sender.sends(plain_message, _to=receiver)

True

In [64]:
cipher_signed_with_noise_bis = sender.sends(plain_message, _to=receiver, _with_noises=True)
cipher_signed_with_noise_bis


'NoIsYnOiSy8QoP!S/SactnrIf3VE89\tc-c&F#;N`iWj|3v"%A6<#T}YB9[[n>RBHIxSQRa~uSR?>gRW%_IJPr3CooA!ad}"5~^_WJFy:\\gFoI0B8,$g<Va\n4}I9"b\n1[PfpEBV>]8@zw64| \'%V!>ylGe6mD_"\njH,ONoIsYnOiSy'

In [65]:
cipher_signed_with_noise_bis[10:-10] # trim NoIsYnOiSy

'8QoP!S/SactnrIf3VE89\tc-c&F#;N`iWj|3v"%A6<#T}YB9[[n>RBHIxSQRa~uSR?>gRW%_IJPr3CooA!ad}"5~^_WJFy:\\gFoI0B8,$g<Va\n4}I9"b\n1[PfpEBV>]8@zw64| \'%V!>ylGe6mD_"\njH,O'

In [66]:
cipher_signed_with_noise == cipher_signed_with_noise_bis[10:-10]

False

##### Q: Do you know why is it `False`?

#### Decryption

In [67]:
signed_message = receiver.decrypt(cipher_signed)
receiver.encrypt(signed_message, key=sender.pub) # remove signature


'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

In [68]:
signed_message_with_noise = receiver.decrypt(cipher_signed_with_noise)
message_with_noise = receiver.encrypt(signed_message_with_noise, key=sender.pub) # unsign
utils.remove_noises(message_with_noise)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

In [69]:
trimmed_cipher_signed_with_noise = cipher_signed_with_noise_bis[10:-10]

signed_message_with_noise = receiver.decrypt(trimmed_cipher_signed_with_noise)
message_with_noise = receiver.encrypt(signed_message_with_noise, key=sender.pub) # unsign

utils.remove_noises(message_with_noise)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

##### Shortcut

In [70]:
receiver.gets(cipher_signed, _from=sender.pub)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

In [71]:
utils.remove_noises(receiver.gets(cipher_signed_with_noise, _from=sender))

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

In [72]:
receiver.gets(cipher_signed_with_noise_bis, _from=sender)

'Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)'

#### Attacks

##### Reading Attack


In [73]:
hacker.decrypt(cipher_signed)


'"a0\t-<}Y3Wkm\'b30v&eDg9ueN{90tk,|qEyNki x!n0Q%?\n:VED$4z>+oNFW(Y}9]1*AV.9F9~Vlr/P'

In [74]:
hacker.encrypt(cipher_signed, key=sender.pub)


'}q_p1OA\'[&d)s%:iq*?C#FDY=nY[k^ve$Obs]E$I6n>$5mv!P(}+cV\'1#]{!ktc_YV1"_l_]\\laYXnt'

In [75]:
hacker.encrypt(cipher_signed, key=receiver.pub)


'0\nfaATRsh"14~IR;,BQ}7kPhpPYn%L$JnrJInJ:U@(iT=e\\@K_Xo}S#w(gjf]zZY<h!\\6\'\'*4#\nhn\''

In [76]:
hacker.sends(cipher_signed, _to=receiver.pub)


'W"v?G]wpS}vlZL\tTHUj8\'G)!b>I#x*-M;W#;kZ/J9Q0D.2,uxr9u!XAz~ ja:u/R{0ep 10&\'\tPxgVCr'

In [77]:
hacker.gets(cipher_signed, _from=sender)

'\n1>02Wvd|#v|,B)\n;[:TqLMKaG>=N|qa*W%$SaO|"Wv%<)D0\t:0]hv1}xu:`Qc:k\'A3+-|et,wLnsCF&'

In [78]:
hacker.gets(cipher_signed, _from=bob)

'W"v?G]wpS}vlZL\tTHUj8\'G)!b>I#x*-M;W#;kZ/J9Q0D.2,uxr9u!XAz~ ja:u/R{0ep 10&\'\tPxgVCr'

##### Sending Attack


In [79]:
print(f"""
Original:
========
{plain_message}

Scam:
=====
{scam_message}
"""
)


Original:
Hello Bob! This is Alice. Please send me $1000 to this account: (1234567890)

Scam:
=====
Hello Bob! This is Alice. Please send me $1000 to this account: (0987654321)



In [80]:
non_signed_scam = hacker.encrypt(scam_message, key=receiver.pub)

receiver.gets(non_signed_scam, _from=sender)

'"b<p-vdWDV&vIO!TJ6<g[ZhkP]{E/\t{f3l_vXxGSkbl^W>~T5d%sFkR9$I_9SYwC ,@Nq&YqdS84rY'

In [81]:
non_signed_scam = hacker.encrypt(scam_message, key=sender.pub)

receiver.gets(non_signed_scam, _from=sender)

'}oZ:l2}zfz6\nWew!t_ \'4WR`P=06v))o-3:DTE#&,[}>KDCK}+)"+1i6lAE+ewaSR:uyP1Ly(>K_H1+'

In [82]:
signed_scam = hacker.sends(scam_message, _to=receiver)

receiver.gets(signed_scam, _from=sender)

'\n8I/suNK:OD^+zEn B|J;Fl?`PC_+]c=MPnR-`nnwh \ta/qO!.;pQs3?+;\'C<WH!@7":,v\'sD]4M5F'

##### Possible Weakness

In [83]:
_, leaked_sender_priv = sender._Node__keys  # key of alice = expected sender

fake_signed_scam = hacker.encrypt(scam_message, leaked_sender_priv)
cipher_fake_signed_scam = hacker.encrypt(fake_signed_scam, receiver.pub)

cipher_fake_signed_scam



'/>?E1(V\tiR!aS%S)R6ZZA*0^paZkKOSDiI,p@gN[.j+WqQtp>7bl\\v_1Wy3\n:@\n(hLoC5Fg}LGD|Dw'

In [84]:
receiver.gets(cipher_fake_signed_scam, _from=sender)

'Hello Bob! This is Alice. Please send me $1000 to this account: (0987654321)'

# Blockchain


In [85]:
from bcbcpy import blockchain

## Recap: Data Structure

### List - Array

### Linked List

## Block

In [86]:
from bcbcpy.blockchain.block import InitialBlock, Block

### Initial Block

In [87]:
init_block = InitialBlock(initial_data="initial data")

In [88]:
init_block.data

'initial data'

In [89]:
init_block.hash

'00002d64bd7129caa2aa8ec458081519ac26034158e60962f6fd1428e22b1036'

In [90]:
init_block

{
    "prev_hash": "",
    "hash": "00002d64bd7129caa2aa8ec458081519ac26034158e60962f6fd1428e22b1036",
    "data": "initial data",
    "nonce": 26206
}

### Block Data

In [91]:
second_block = Block(data="second data", prev_block= init_block)
second_block

{
    "prev_hash": "00002d64bd7129caa2aa8ec458081519ac26034158e60962f6fd1428e22b1036",
    "hash": "ac19b9c98cc9a1655bd66662c552b8667c2140a451ce635b24aa2fdf34555aea",
    "data": "second data",
    "nonce": 0
}

In [92]:
second_block.is_valid()

False

In [93]:
second_block.mine()
second_block

{
    "prev_hash": "00002d64bd7129caa2aa8ec458081519ac26034158e60962f6fd1428e22b1036",
    "hash": "0000f1a7cc9b3335858fdc615b274e07568931de65e82fb991ac257e168b89c1",
    "data": "second data",
    "nonce": 12005
}

In [94]:
second_block.is_valid()

True

In [95]:
third_block = Block("third data", prev_block= second_block)
third_block

{
    "prev_hash": "0000f1a7cc9b3335858fdc615b274e07568931de65e82fb991ac257e168b89c1",
    "hash": "f94643bbf3edffb31b549ce63a5adbd4542b7e3b6d45f32ae69da4f9bc87b6f7",
    "data": "third data",
    "nonce": 0
}

In [96]:
third_block.mine()
third_block

{
    "prev_hash": "0000f1a7cc9b3335858fdc615b274e07568931de65e82fb991ac257e168b89c1",
    "hash": "0000eecdfce497ff979154b54ddb7f8fe0114683c0102afb900cd93497951e80",
    "data": "third data",
    "nonce": 2917
}

In [97]:
forth_block = Block("forth data", third_block)
forth_block.mine()
forth_block

{
    "prev_hash": "0000eecdfce497ff979154b54ddb7f8fe0114683c0102afb900cd93497951e80",
    "hash": "00003e75b774fcf431090692c595e06fa5d05bc7730a63673e4e4304eb4faf6a",
    "data": "forth data",
    "nonce": 101056
}

In [98]:
fifth_block = Block("fifth block", forth_block)
fifth_block.mine()
fifth_block

{
    "prev_hash": "00003e75b774fcf431090692c595e06fa5d05bc7730a63673e4e4304eb4faf6a",
    "hash": "0000d8cb89717a6b1904bf4f44aab11529b3ee378d0256620f608c3dd73694b9",
    "data": "fifth block",
    "nonce": 34705
}

In [99]:
init_block.is_valid(), second_block.is_valid(), third_block.is_valid(), forth_block.is_valid(), fifth_block.is_valid()

(True, True, True, True, True)

### Making some change in past data

In [100]:
third_block.data = "not third data"
third_block

{
    "prev_hash": "0000f1a7cc9b3335858fdc615b274e07568931de65e82fb991ac257e168b89c1",
    "hash": "66ad1a8ee995ddbcaaf131585222c10a396a960f0c441578d5613417fa5d107a",
    "data": "not third data",
    "nonce": 2917
}

In [101]:
init_block.is_valid(), second_block.is_valid(), third_block.is_valid(), forth_block.is_valid(), fifth_block.is_valid()

(True, True, False, False, False)

In [102]:
third_block.data = "third data"
init_block.is_valid(), second_block.is_valid(), third_block.is_valid(), forth_block.is_valid(), fifth_block.is_valid()

(True, True, True, True, True)

In [103]:
third_block.data = "not third data"
third_block.mine()
third_block.is_valid()

True

In [104]:
init_block.is_valid(), second_block.is_valid(), third_block.is_valid(), forth_block.is_valid(), fifth_block.is_valid()

(True, True, True, False, False)

In [105]:
third_block.data = "third data"
third_block.mine()
init_block.is_valid(), second_block.is_valid(), third_block.is_valid(), forth_block.is_valid(), fifth_block.is_valid()

(True, True, True, True, True)

## Blockchain

A submodule is available at 
```python
from bcbcpy.blockchain import chain
```

### Root

In [106]:
blockchain.RootChain()

{
    "root_block": {
        "hash": "0000a456e7b5a5eb059e721fb431436883143101275c4077f83fe70298f5623d",
        "data": ""
    }
}

In [107]:
initial_data = {"INITIAL AGREEMENT": {"Bob": 1000, "Alice": 500}}

initial_data_string = utils.obj2txt(initial_data, indent=4)
root_chain = blockchain.RootChain(initial_data_string, difficulty=4)

root_chain

{
    "root_block": {
        "hash": "0000579da420f2b62497716cc60e95a0f7ad46336c42062b7485a27fd91d99f0",
        "data": "{\n    \"INITIAL ..."
    }
}

### Chain

In [108]:
prev_block = root_chain.last_block
new_block = Block("new block", prev_block)
new_block

{
    "prev_hash": "0000579da420f2b62497716cc60e95a0f7ad46336c42062b7485a27fd91d99f0",
    "hash": "73095d20a662a26d26eec210c0dc51c34816098cdab05f941487ee5e8133d6ad",
    "data": "new block",
    "nonce": 0
}

---

In [109]:
root_chain.add_block(
  new_block
)


AssertionError: Block is not valid.

In [110]:
new_block.mine() # validation
new_block

{
    "prev_hash": "0000579da420f2b62497716cc60e95a0f7ad46336c42062b7485a27fd91d99f0",
    "hash": "00007b581d241fea3fb8bf4c7c5c1941af9abb3a5ce05abb97b4521d34bab727",
    "data": "new block",
    "nonce": 2823
}

In [111]:
root_chain.add_block(
    new_block
)

In [112]:
root_chain

{
    "root_block": {
        "hash": "0000579da420f2b62497716cc60e95a0f7ad46336c42062b7485a27fd91d99f0",
        "data": "{\n    \"INITIAL ..."
    },
    "last_block": {
        "hash": "00007b581d241fea3fb8bf4c7c5c1941af9abb3a5ce05abb97b4521d34bab727",
        "data": "new block"
    }
}

In [113]:
chain = blockchain.Chain(root_chain=root_chain, info="local network")
chain

{
    "root_block": {
        "hash": "000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9",
        "data": "local network"
    }
}

## Transaction


In [114]:
from bcbcpy.blockchain.block import transaction

### Nodes setup

In [115]:
from bcbcpy.blockchain.node import Node

alice = Node(crypto.RSAPairKeys.generate_pairs(), "Alice")
bob = Node(crypto.RSAPairKeys.generate_pairs(), "Bob")
other_person_in_the_network = Node(crypto.RSAPairKeys.generate_pairs())

### Transaction Data

In [116]:
assets = 1000
sender = bob
receiver = alice

In [117]:
receiver_id = receiver.id
receiver_pub = receiver.pub

prev_block = chain.last_block

In [118]:
transaction_data = transaction.TransactionData(
    assets, sender, receiver_id, receiver_pub, prev_block
)

print(transaction_data)

{'timestamp': '2023-05-09T22:01:00.655549', 'sender_id': 'Bob', 'receiver_id': 'Alice', 'assets': 1000, 'prev_hash': '000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9', 'sender_confirmation': ')u9>%>prl0YwZU:X0YA*Pr}fF?(zSv@Y\\BJ{kiDg4tX[oQ^z4JF}XsU+Pb>\\q)V&8Iet~<J{4|K*j%P"J\tY5Bm\toAF_ov`5uj41;9!86B$gH:vqTkpj,xpd|iw3}JbUs95&$1[P'}


### Transaction Block

In [119]:
transaction_block = blockchain.TransactionBlock(transaction_data)
transaction_block

{
    "prev_hash": "000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9",
    "hash": "5fa1e4b30e322577ffe63903a4f0bbc7b5a0646342ef6fac02b1ddc076e306b9",
    "data": "{'timestamp': '2023-05-09T22:01:00.655549', 'sender_id': 'Bob', 'receiver_id': 'Alice', 'assets': 1000, 'prev_hash': '000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9', 'sender_confirmation': ')u9>%>prl0YwZU:X0YA*Pr}fF?(zSv@Y\\\\BJ{kiDg4tX[oQ^z4JF}XsU+Pb>\\\\q)V&8Iet~<J{4|K*j%P\"J\\tY5Bm\\toAF_ov`5uj41;9!86B$gH:vqTkpj,xpd|iw3}JbUs95&$1[P'}",
    "nonce": 0
}

### Verification


#### Anyone in the network

In [121]:
data = transaction_block.data

print(data)


{'timestamp': '2023-05-09T22:01:00.655549', 'sender_id': 'Bob', 'receiver_id': 'Alice', 'assets': 1000, 'prev_hash': '000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9', 'sender_confirmation': ')u9>%>prl0YwZU:X0YA*Pr}fF?(zSv@Y\\BJ{kiDg4tX[oQ^z4JF}XsU+Pb>\\q)V&8Iet~<J{4|K*j%P"J\tY5Bm\toAF_ov`5uj41;9!86B$gH:vqTkpj,xpd|iw3}JbUs95&$1[P'}


In [122]:
sender_confirmation=""")u9>%>prl0YwZU:X0YA*Pr}fF?(zSv@Y\\BJ{kiDg4tX[oQ^z4JF}XsU+Pb>\\q)V&8Iet~<J{4|K*j%P"J\tY5Bm\toAF_ov`5uj41;9!86B$gH:vqTkpj,xpd|iw3}JbUs95&$1[P""" 
print(sender_confirmation)

)u9>%>prl0YwZU:X0YA*Pr}fF?(zSv@Y\BJ{kiDg4tX[oQ^z4JF}XsU+Pb>\q)V&8Iet~<J{4|K*j%P"J	Y5Bm	oAF_ov`5uj41;9!86B$gH:vqTkpj,xpd|iw3}JbUs95&$1[P


In [123]:
confirmation_string = other_person_in_the_network.encrypt(sender_confirmation, key= sender.pub)

print(confirmation_string)

[['assets',	1000],
['receiver_pub',	(34049, 16153)],
['prev_hash',	000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9]]


In [124]:
print(receiver.pub)
print(chain.hash)

(34049, 16153)
000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9


### Block Validation

In [125]:
transaction_block.is_valid()

False

In [126]:
transaction_block.mine()
transaction_block

{
    "prev_hash": "000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9",
    "hash": "000036ebb76fbf1e470018f63d363f24f064c5d30121d726ca474f1499dca7c0",
    "data": "{'timestamp': '2023-05-09T22:01:00.655549', 'sender_id': 'Bob', 'receiver_id': 'Alice', 'assets': 1000, 'prev_hash': '000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9', 'sender_confirmation': ')u9>%>prl0YwZU:X0YA*Pr}fF?(zSv@Y\\\\BJ{kiDg4tX[oQ^z4JF}XsU+Pb>\\\\q)V&8Iet~<J{4|K*j%P\"J\\tY5Bm\\toAF_ov`5uj41;9!86B$gH:vqTkpj,xpd|iw3}JbUs95&$1[P'}",
    "nonce": 62310
}

In [127]:
transaction_block.is_valid()

True

### Adding to Chain

In [128]:
chain.add_block(transaction_block)
chain

{
    "root_block": {
        "hash": "000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9",
        "data": "local network"
    },
    "last_block": {
        "hash": "000036ebb76fbf1e470018f63d363f24f064c5d30121d726ca474f1499dca7c0",
        "data": "{'timestamp': '..."
    }
}

## Second Transaction


In [129]:
assets = 500
sender = alice
receiver = bob
prev_block = chain.last_block

In [130]:
transaction_block = sender.make_transaction_block(
    assets=assets,
    to=receiver,
    prev_block=prev_block,
)

### Verification ...

In [131]:
data = transaction_block.data
print(data)

{'timestamp': '2023-05-09T22:02:06.249209', 'sender_id': 'Alice', 'receiver_id': 'Bob', 'assets': 500, 'prev_hash': '000036ebb76fbf1e470018f63d363f24f064c5d30121d726ca474f1499dca7c0', 'sender_confirmation': '\nu&R|YfX# Ta@W_.>"?M3#c[@y\nxxgZYma^^7c`mjTS2VT^+x)\nA/NETWY&2}]es^fVl={?aF=jo]@0?{(\tKX<J^W1#S2@1Edt!2*\nVJ>Bzk(Wuk,x~j p5/0Y?DlXF,X-m '}


In [132]:
sender_confirmation="""\nu&R|YfX# Ta@W_.>"?M3#c[@y\nxxgZYma^^7c`mjTS2VT^+x)\nA/NETWY&2}]es^fVl={?aF=jo]@0?{(\tKX<J^W1#S2@1Edt!2*\nVJ>Bzk(Wuk,x~j p5/0Y?DlXF,X-m """ 
confirmation_string = other_person_in_the_network.encrypt(sender_confirmation, key= sender.pub)

confirmation = utils.txt2obj(confirmation_string)
print(confirmation)

[['assets',	500],
['receiver_pub',	(34691, 16733)],
['prev_hash',	000036ebb76fbf1e470018f63d363f24f064c5d30121d726ca474f1499dca7c0]]


In [133]:
print(receiver.pub)
print(chain.hash)

(34691, 16733)
000036ebb76fbf1e470018f63d363f24f064c5d30121d726ca474f1499dca7c0


### Adding to Chain

In [134]:
transaction_block.mine() # validation
chain.add_block(transaction_block)

In [135]:
chain

{
    "root_block": {
        "hash": "000015420debcdd55c00babc9c3a5a9c4ab22280cf9a6874986c427281dd79b9",
        "data": "local network"
    },
    "...": "...",
    "last_block": {
        "hash": "0000400eefda387a2c3c7ef8ffc4ad50e5f8837109c096ad1c08032edc219e89",
        "data": "{'timestamp': '..."
    }
}